In [75]:
import numpy as np
import cv2
import glob
import dlib
import math
import itertools
import pickle
import pandas as pd
import os
from math import sqrt

In [76]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/Users/guvenerkaya/Downloads/dlib_shape_predictor") 

data = {}

In [26]:
def get_landmarks(image):
    data_points = []
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
    clahe_image = clahe.apply(gray) #Contrast Limited Adaptive Histogram Equalization
    detections = detector(gray, 1)
    if len(detections) > 0:
        for k,d in enumerate(detections): #For all detected face instances individually
            shape = predictor(image, d) #Draw facial landmarks with dlib's predictor class
            xlist = []
            ylist = []
            for i in range(0,68): #Store X and Y coordinates of the facial landmarks in two lists
                xlist.append(float(shape.part(i).x))
                ylist.append(float(shape.part(i).y))
            landmarks = []
            for x, y in zip(xlist, ylist):
                landmarks.append(x) # get them in order in the same list wit(x_1,y_1,x_2,y_2...)
                landmarks.append(y)
            data['landmarks'] = landmarks 
            data_points.append(data['landmarks'])
        return data_points
    else:
        return None

In [ ]:
key_landmarks = []
no_face_detected = []
detected_faces = []
video_num = []
keyframe_num =[]

files = glob.glob('/Volumes/LIRIS_ACCEDE/keyframes 2/*.png') #read all extracted key frames
for file in files:
    
#image = cv2.imread('/Volumes/LIRIS_ACCEDE/keyframes 2/ACCEDE00195_03.png') # read image/frame
    image = cv2.imread(file) #store each key frame one by one to draw facial landmarks
    data_points = get_landmarks(image) #store facial landmarks 
    if data_points != None: #if face is found
        for i in range(len(data_points)): #adding the facial landmarks of each found face to the list key_landmarks
            key_landmarks.append(data_points[i])
            video_number = file[40:45] #detecting which video the key frames belong 
            video_num.append(video_number)
            keyframe_number = file[46:48]#detecting the which keyframe of the video it is
            keyframe_num.append(keyframe_number)
            detected_faces.append(i+1)#detecting the number of detected faces in the key frame
         
    #else:
      #  no_face_detected.append(file[34:])


#df1 = pd.DataFrame(data_points, columns = name)
#print(len(files))    
#print(files)
#set column names
name_x = np.arange(1, 69, 1)
name_x = list(map(str, name_x))
name_x = ["x_" + item for item in name_x]

name_y = np.arange(1, 69, 1)
name_y = list(map(str, name_y))
name_y = ["y_" + item for item in name_y]

name = [None]*(len(name_x)+len(name_y))
name[::2] = name_x
name[1::2] = name_y

df = pd.DataFrame(key_landmarks, index = video_num,   columns=name)
df.set_index( pd.Index(keyframe_num), append=True, inplace=True)
df.set_index(pd.Index(detected_faces), append = True, inplace= True)
df.index.names = ['id','keyframe','face']
df_original = df
df
#video_num
#keyframe_num
#pickle.dump(df, open('/Users/guvenerkaya/Desktop/datasets_and_models/kf_facial_landmarks', 'wb')) # save key frames with drawn facial landmarks




In [ ]:
df = df.values

final_data = []
#calculate the euclidean distances of facial landarks of the key frames
for row in range(0, len(df)):
    data = df[row]
    #print(data)
    print("\n")
    x = []
    y = []
    new_data = []
    x = data[::2]
    y = data[1::2]
    #print(x)
    #print("\n")
    #print(y)
    #print("\n")
    for i in range(0, 68):
        for j in range(i+1, 68):
            distance = sqrt((x[i] - x[j]) * (x[i] - x[j]) + (y[i] - y[j]) * (y[i] - y[j])) 
            new_data.append(distance)
    final_data.append(new_data)
#set column names
names = []
for i in range(1, 69, 1):
    for j in range(i+1, 69, 1):
        name = "distance" + str(i) + "<->" + str(j)
        names.append(name)

df_new = pd.DataFrame(final_data, columns=names, index=df_original.index)
#extend the data frame with euclidean distances
df3 = pd.concat([df_original, df_new], axis=1, sort=False)
df3

In [29]:
best = ['distance64<->66', 'distance25<->44', 'distance45<->47', 'distance65<->68', 'distance44<->48', 'distance20<->39', 'distance63<->67', 'distance34<->63', 'distance62<->68', 'distance60<->62', 'distance21<->39', 'distance21<->41', 'distance52<->67', 'distance54<->56', 'distance54<->68', 'distance21<->40', 'distance49<->65', 'distance53<->57', 'distance20<->42', 'distance10<->55', 'distance36<->57', 'distance53<->66', 'distance49<->58', 'distance24<->43', 'distance57<->64', 'distance9<->61', 'distance64<->67', 'distance50<->68', 'distance49<->59', 'distance25<->47', 'distance32<->34', 'distance31<->63', 'distance39<->42', 'distance49<->60', 'distance59<->62', 'distance60<->61', 'distance51<->53', 'distance35<->55', 'distance38<->42', 'distance49<->56', 'distance50<->57', 'distance63<->65', 'distance31<->62', 'distance21<->42', 'distance32<->61', 'distance20<->41', 'distance54<->61', 'distance58<->63', 'distance10<->65', 'distance9<->59']
df4 = df3[best] 
df4

distance64<->66  distance25<->44  distance45<->47  \
id    keyframe face                                                      
00001 01       1            6.000000        31.240999        13.000000   
      02       1            1.000000        39.812058         5.385165   
00003 02       1            4.123106        14.317821         6.000000   
00004 02       1            9.486833        32.310989        11.180340   
00005 02       1           15.033296        52.000000        18.248288   
00007 03       1            2.236068        32.249031        12.041595   
00011 03       1           22.090722        27.802878        14.035669   
00012 01       1            6.082763        12.649111         2.236068   
00013 02       1           20.024984        28.844410        10.198039   
00014 01       1            2.236068         5.656854         3.000000   
00015 01       1            5.099020        40.496913         8.062258   
00017 01       1            3.605551        31.064449         8.944272   
00021 01       1            2.828427        25.942244        11.000000   
      02       1            1.000000        16.970563         7.000000   
00022 01       1            5.000000        20.591260        10.049876   
00023 02       1            1.414214        26.400758        10.198039   
00025 02       1           17.029386        33.241540         7.071068   
      05       1            2.000000        17.204651         8.062258   
               2           16.031220        28.861739         7.000000   
      06       1            0.000000        20.615528         9.055385   
00026 01       1            6.000000        33.241540        10.049876   
00028 01       1            1.000000        29.614186        11.180340   
00029 01       1            0.000000         5.385165         3.162278   
00030 01       1            1.414214        20.615528         7.000000   
      02       1            2.236068        50.695167        13.038405   
00036 01       1            5.000000        49.396356         9.486833   
      02       1            2.236068        30.000000        13.152946   
00037 01       1            4.123106        27.202941         7.071068   
00038 02       1            2.236068        65.734314        22.090722   
00039 01       1            7.280110        65.368188        21.000000   
...                              ...              ...              ...   
09771 04       1            5.830952        45.486262        24.020824   
09772 01       1            1.414214         5.385165         2.000000   
09774 03       1           11.180340        25.317978         4.123106   
               2            1.000000        11.313708         8.000000   
09775 02       1            9.055385        32.557641        10.198039   
09776 01       1            6.403124        36.345564        10.630146   
      02       1           34.132096        98.234414        13.038405   
09777 01       1            7.211103        41.109610         8.485281   
09778 03       1            1.000000         9.000000         2.828427   
09779 01       1            1.000000        11.180340         6.082763   
09780 02       1            2.236068        25.179357         5.830952   
09783 02       1           15.132746        24.083189         8.062258   
09784 02       1            4.123106        60.638272        13.152946   
09785 01       1            1.414214        27.202941         9.000000   
      02       1            5.000000        92.720009        28.071338   
09786 01       1            3.605551        28.071338         9.848858   
      02       1            5.000000        53.235327        15.652476   
      03       1            4.123106        88.323270        28.000000   
09790 02       1            5.830952        17.117243         4.472136   
09791 02       1            5.000000        16.124515         7.071068   
      03       1            1.414214        12.206556         5.385165   
09794 01       1         

In [13]:
#pickle.dump(df4, open('/Users/guvenerkaya/Desktop/datasets_and_models/kf_50_best_distances', 'wb'))
#svm_trained = pickle.load(open('/Users/guvenerkaya/Desktop/datasets_and_models/SVM_trained', 'rb'))
df5 = pd.read_pickle('/Users/guvenerkaya/Desktop/datasets_and_models/kf_50_best_distances')
df5

distance64<->66  distance25<->44  distance45<->47  \
id    keyframe face                                                      
00001 01       1            6.000000        31.240999        13.000000   
      02       1            1.000000        39.812058         5.385165   
00003 02       1            4.123106        14.317821         6.000000   
00004 02       1            9.486833        32.310989        11.180340   
00005 02       1           15.033296        52.000000        18.248288   
00007 03       1            2.236068        32.249031        12.041595   
00011 03       1           22.090722        27.802878        14.035669   
00012 01       1            6.082763        12.649111         2.236068   
00013 02       1           20.024984        28.844410        10.198039   
00014 01       1            2.236068         5.656854         3.000000   
00015 01       1            5.099020        40.496913         8.062258   
00017 01       1            3.605551        31.064449         8.944272   
00021 01       1            2.828427        25.942244        11.000000   
      02       1            1.000000        16.970563         7.000000   
00022 01       1            5.000000        20.591260        10.049876   
00023 02       1            1.414214        26.400758        10.198039   
00025 02       1           17.029386        33.241540         7.071068   
      05       1            2.000000        17.204651         8.062258   
               2           16.031220        28.861739         7.000000   
      06       1            0.000000        20.615528         9.055385   
00026 01       1            6.000000        33.241540        10.049876   
00028 01       1            1.000000        29.614186        11.180340   
00029 01       1            0.000000         5.385165         3.162278   
00030 01       1            1.414214        20.615528         7.000000   
      02       1            2.236068        50.695167        13.038405   
00036 01       1            5.000000        49.396356         9.486833   
      02       1            2.236068        30.000000        13.152946   
00037 01       1            4.123106        27.202941         7.071068   
00038 02       1            2.236068        65.734314        22.090722   
00039 01       1            7.280110        65.368188        21.000000   
...                              ...              ...              ...   
09771 04       1            5.830952        45.486262        24.020824   
09772 01       1            1.414214         5.385165         2.000000   
09774 03       1           11.180340        25.317978         4.123106   
               2            1.000000        11.313708         8.000000   
09775 02       1            9.055385        32.557641        10.198039   
09776 01       1            6.403124        36.345564        10.630146   
      02       1           34.132096        98.234414        13.038405   
09777 01       1            7.211103        41.109610         8.485281   
09778 03       1            1.000000         9.000000         2.828427   
09779 01       1            1.000000        11.180340         6.082763   
09780 02       1            2.236068        25.179357         5.830952   
09783 02       1           15.132746        24.083189         8.062258   
09784 02       1            4.123106        60.638272        13.152946   
09785 01       1            1.414214        27.202941         9.000000   
      02       1            5.000000        92.720009        28.071338   
09786 01       1            3.605551        28.071338         9.848858   
      02       1            5.000000        53.235327        15.652476   
      03       1            4.123106        88.323270        28.000000   
09790 02       1            5.830952        17.117243         4.472136   
09791 02       1            5.000000        16.124515         7.071068   
      03       1            1.414214        12.206556         5.385165   
09794 01       1         

In [14]:
SVM = pickle.load(open('/Users/guvenerkaya/Desktop/datasets_and_models/SVM_trained', 'rb'))
test  = SVM.predict_proba(df5)    
#test = SVM.predict(df5)
a =pd.DataFrame(test)
#a
x = df5.index
a.index = x
SVM.classes_

array(['angry', 'contemptuous', 'disgusted', 'fearful', 'happy',
       'neutral', 'sad', 'surprised'], dtype=object)

In [15]:
a.columns = ['angry', 'contemptuous', 'disgusted', 'fearful' ,'happy', 'neutral' ,'sad', 'surprised']
#by_row_index = a.groupby(a.index)
#df_means = by_row_index.mean(level='id')
df_means= a.mean(level='id')
index = df_means.index.to_list

index_int = df_means.index.astype(str).astype(int)
df_means.index = index_int
df_means

,angry,contemptuous,disgusted,fearful,happy,neutral,sad,surprised
id,,,,,,,,
1,0.005927,0.031134,0.015938,0.002207,0.191053,0.725753,0.012055,0.015932
3,0.997403,0.001097,0.000223,0.000039,0.000013,0.000095,0.001117,0.000013
4,0.131108,0.017808,0.634444,0.006901,0.018430,0.153751,0.031554,0.006004
5,0.016743,0.020387,0.011158,0.194806,0.024457,0.413833,0.098020,0.220596
7,0.528609,0.013154,0.003430,0.000907,0.004937,0.444814,0.004045,0.000105
11,0.455420,0.186728,0.017797,0.124830,0.044631,0.020878,0.097670,0.052047
12,0.996849,0.001602,0.000100,0.000078,0.000028,0.000016,0.001281,0.000046
13,0.751001,0.209550,0.000987,0.018329,0.001308,0.000593,0.002559,0.015671
14,0.996794,0.001789,0.000035,0.000127,0.000066,0.000012,0.001080,0.000095


In [39]:
def find_missing(lst): 
    return [x for x in range(lst[0], lst[-1]+1)  
                               if x not in lst] 
feature_list=['angry', 'contemptuous', 'disgusted' ,'fearful','happy', 'neutral', 'sad',
 'surprised']
zero_data = np.zeros(shape=(4534,len(feature_list)))
feature_list=['angry', 'contemptuous', 'disgusted' ,'fearful','happy', 'neutral', 'sad',
 'surprised']

index_list = find_missing(index_int)
df0 = pd.DataFrame(zero_data,index =index_list , columns=feature_list)
df0
last_df = df_means.append(df0,ignore_index=False)
last_df.index.name = 'id'
last_df
df_sorted = last_df.sort_values('id', axis=0)
last_row = [0,0,0,0,0,0,0,0]
add_last_row =  pd.DataFrame({'angry':0, 'contemptuous':0, 'disgusted':0, 'fearful':0, 'happy':0, 'neutral':0, 'sad':0, 'surprised':0}, index =[0])
#last_of_us = pd.concat([add_last_row, df_sorted],)
#pickle.dump(last_of_us, open('/Users/guvenerkaya/Desktop/datasets_and_models/emotion_proabilities', 'wb'))
#index_list[-1]
last_of_us = pd.concat([add_last_row,df_sorted],  ignore_index=True)


In [40]:
last_of_us
pickle.dump(last_of_us, open('/Users/guvenerkaya/Desktop/datasets_and_models/emotion_probabilities', 'wb'))

,angry,contemptuous,disgusted,fearful,happy,neutral,sad,surprised
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.005927,0.031134,0.015938,0.002207,0.191053,0.725753,0.012055,0.015932
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.997403,0.001097,0.000223,0.000039,0.000013,0.000095,0.001117,0.000013
4,0.131108,0.017808,0.634444,0.006901,0.018430,0.153751,0.031554,0.006004
5,0.016743,0.020387,0.011158,0.194806,0.024457,0.413833,0.098020,0.220596
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.528609,0.013154,0.003430,0.000907,0.004937,0.444814,0.004045,0.000105
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [46]:
arousal_data = pd.read_pickle('/Users/guvenerkaya/Desktop/datasets_and_models/arousal_dataframe')
arousal_features = arousal_data.iloc[:,:-1]
#arousal_data.join(last_of_us)
#arousal_features
#last_of_us.index.astype(str).astype(object)
#arousal_emotions
arousal_new = arousal_data.reset_index(drop=True)
arousal_new.iloc[-1]
concencated = arousal_new.join(last_of_us)
final= concencated.drop(labels='arousalValue', axis = 1)
#concencated.insert(32, 'arousalValue',arousal_new.iloc[-1])
final.insert(31, 'arousalValue',arousal_new.iloc[:,-1])
final


,colorfulness,minEnergy,alpha,lightning,globalActivity,nbWhiteFrames,nbSceneCuts,cutLength,flatness_env,wtf_max2stdratio_1,...,slope,angry,contemptuous,disgusted,fearful,happy,neutral,sad,surprised,arousalValue
0,47.939117,0.540554,5,1.057049,0.100531,0,1,117.000000,0.005686,5.41,...,1.414934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.366938
1,52.748894,0.798568,41,0.464044,0.062712,0,1,153.000000,0.000000,5.42,...,1.354866,0.005927,0.031134,0.015938,0.002207,0.191053,0.725753,0.012055,0.015932,2.792376
2,60.224403,0.833286,50,0.294487,0.322534,0,3,52.000000,0.000000,5.18,...,1.312889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.840160
3,82.669434,1.017838,43,0.447583,0.705550,0,1,52.000000,0.000000,5.14,...,1.334575,0.997403,0.001097,0.000223,0.000039,0.000013,0.000095,0.001117,0.000013,2.470821
4,64.427177,0.670391,38,0.059518,0.127405,0,1,117.000000,0.000000,5.09,...,1.390362,0.131108,0.017808,0.634444,0.006901,0.018430,0.153751,0.031554,0.006004,2.082151
5,59.896679,0.852062,31,0.148373,0.106458,0,1,152.000000,0.000000,5.14,...,1.436904,0.016743,0.020387,0.011158,0.194806,0.024457,0.413833,0.098020,0.220596,1.684790
6,44.781807,0.956567,48,1.401878,0.068769,0,1,141.000000,0.000000,5.20,...,1.369900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.721086
7,56.581673,0.945640,42,0.071817,0.195562,0,3,63.000000,0.036094,5.10,...,1.387032,0.528609,0.013154,0.003430,0.000907,0.004937,0.444814,0.004045,0.000105,1.646770
8,66.858261,0.661873,39,0.155335,0.584721,0,0,0.000000,0.000000,5.08,...,1.367238,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.919416
9,57.232445,0.767595,32,0.204125,0.122521,0,0,0.000000,0.000000,5.19,...,1.404063,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.695038


In [73]:
y= final.iloc[:,31]
X=final.iloc[:,:31]
X


,colorfulness,minEnergy,alpha,lightning,globalActivity,nbWhiteFrames,nbSceneCuts,cutLength,flatness_env,wtf_max2stdratio_1,...,medianLightness,slope,angry,contemptuous,disgusted,fearful,happy,neutral,sad,surprised
0,47.939117,0.540554,5,1.057049,0.100531,0,1,117.000000,0.005686,5.41,...,0.431373,1.414934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,52.748894,0.798568,41,0.464044,0.062712,0,1,153.000000,0.000000,5.42,...,0.219608,1.354866,0.005927,0.031134,0.015938,0.002207,0.191053,0.725753,0.012055,0.015932
2,60.224403,0.833286,50,0.294487,0.322534,0,3,52.000000,0.000000,5.18,...,0.400000,1.312889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,82.669434,1.017838,43,0.447583,0.705550,0,1,52.000000,0.000000,5.14,...,0.521569,1.334575,0.997403,0.001097,0.000223,0.000039,0.000013,0.000095,0.001117,0.000013
4,64.427177,0.670391,38,0.059518,0.127405,0,1,117.000000,0.000000,5.09,...,0.474510,1.390362,0.131108,0.017808,0.634444,0.006901,0.018430,0.153751,0.031554,0.006004
5,59.896679,0.852062,31,0.148373,0.106458,0,1,152.000000,0.000000,5.14,...,0.470588,1.436904,0.016743,0.020387,0.011158,0.194806,0.024457,0.413833,0.098020,0.220596
6,44.781807,0.956567,48,1.401878,0.068769,0,1,141.000000,0.000000,5.20,...,0.325490,1.369900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,56.581673,0.945640,42,0.071817,0.195562,0,3,63.000000,0.036094,5.10,...,0.431373,1.387032,0.528609,0.013154,0.003430,0.000907,0.004937,0.444814,0.004045,0.000105
8,66.858261,0.661873,39,0.155335,0.584721,0,0,0.000000,0.000000,5.08,...,0.309804,1.367238,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,57.232445,0.767595,32,0.204125,0.122521,0,0,0.000000,0.000000,5.19,...,0.454902,1.404063,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [71]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.24, random_state=0)
lr= LinearRegression()
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [74]:
y_pred = lr.predict(X_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)
import scipy
scipy.stats.pearsonr(y_test, y_pred)

(0.23341817528709355, 1.8029787222485446e-30)

In [325]:
arousal_X = arousal_data.iloc[:,:23]
arousal_y = arousal_data.iloc[:,23]
X_train1, X_test1, y_train1, y_test1 = train_test_split(arousal_X, arousal_y, test_size=0.25, random_state=0)
lr_1 = LinearRegression()
lr_1.fit(X_train1,y_train1)
y_pred1 = lr_1.predict(X_test1)
r2_score(y_test1,y_pred1)

0.05562799891091974